# Sphärisches Pendel

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Als Verallgemeinerung des mathematischen Pendels wird nun das sphärische Pendel betrachtet. Bei der numerischen Lösung wird ausgenutzt, dass die Komponente $L_z$ des Drehimpulses erhalten ist und sich somit die Zahl der Differentialgleichungen 1. Ordnung um eins reduzieren lässt. Die verbleibenden drei Differentialgleichungen lauten dann in dimensionsloser Form

\begin{align}
\frac{\text{d}\varphi}{\text{d}t} &= \frac{L_z}{\sin^2(\theta)}\\
\frac{\text{d}\theta}{\text{d}t} &= v_\theta\\
\frac{\text{d}v_\theta}{\text{d}t} &= L_z^2\frac{\cos(\theta)}{\sin^3(\theta)} - \sin(\theta)
\end{align}

### Importanweisungen

In [ ]:
from math import cos, sin
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Implementierung der Differentialgleichung

Die nachfolgende Routine stellt die Differentialgleichung auf.

In [ ]:
def dy_dt(t, y, l_z):
    phi, theta, v_theta = y
    return (l_z/sin(theta)**2, v_theta,
            l_z**2*cos(theta)/sin(theta)**3 - sin(theta))

### Lösung des Differentialgleichungssystems

Das Differentialgleichungssystems wird wieder mit der SciPy-Funktion `integrate.solve_ivp` gelöst, wobei nun die $z$-Komponente des Drehimpulses als Tupel im Argument `args` übergeben wird. Die Fehlerschranken für den absoluten und relativen Fehler sind auf feste Werte gesetzt.

Zurückgegeben werden neben den Zeitpunkten die zugehörigen Werte der Winkel $\varphi$ und $\theta$, die Winkelgeschwindigkeit $v_\theta$ sowie die Anzahl der Aufrufe von `dy_dt`.

In [ ]:
def trajectory(t_end, n_out, phi_0, theta_0, v_theta_0,
               l_z):
    t_eval = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(
        dy_dt, (0, t_end), [phi_0, theta_0, v_theta_0],
        args=(l_z,), t_eval=t_eval, atol=1e-8, rtol=1e-8)
    phi, theta, v_theta = solution.y[:]
    t = solution.t
    return t, phi, theta, v_theta, solution.nfev

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Die folgenden Parameter können mit Hilfe von Schiebereglern eingestellt werden:
- `t_end`: Länge des Zeitintervalls
- `n_out`: Anzahl der Zeitpunkte
- `v_phi_0`: Anfangswinkelgeschwindigkeit des Azimutwinkels
- `theta_0`: anfänglicher Polarwinkel
- `v_theta_0`: Anfangswinkelgeschwindigkeit des Polarwinkels

Der Anfangswinkel $\varphi(0)$ wird in der Funktion `plot_result` auf null gesetzt.

Die graphische Darstellung der Bewegung erfolgt durch Projektion der Bahnkurve auf die $x$-$y$-Ebene.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=75, min=10, max=300, step=1,
                   description=r"$\tau_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=1000, min=100, max=5000, step=100,
                   description=r"$n_\text{out}$"),
               "v_phi_0":
               widgets.FloatSlider(
                   value=0.1, min=-1, max=1, step=0.1,
                   description=r"$v_\varphi(0)$"),
               "theta_0":
               widgets.FloatSlider(
                   value=2.6, min=0.1, max=3.14, step=0.1,
                   description=r"$\theta(0)$"),
               "v_theta_0":
               widgets.FloatSlider(
                   value=0, min=-2.5, max=2.5, step=0.1,
                   description=r"$v_\theta(0)$")
               }

@interact(**widget_dict)
def plot_result(t_end, n_out, v_phi_0, theta_0, v_theta_0):
    l_z = v_phi_0 * sin(theta_0)**2
    phi_0 = 0
    t, phi, theta, v_theta, count = trajectory(
        t_end, n_out, phi_0, theta_0, v_theta_0, l_z)

    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    ax.plot(np.cos(phi)*np.sin(theta),
            np.sin(phi)*np.sin(theta))
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    print(f"{count} Aufrufe von dy_dt")